In [1]:
import pandas as pd

# Use raw string (r"") or double backslashes
df = pd.read_csv(r"D:\Guvi-1st Project\traffic_stops - traffic_stops_with_vehicle_number.csv")
df.head()


C:\Users\mpkol\AppData\Local\Temp\ipykernel_584\3344252902.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\Guvi-1st Project\traffic_stops - traffic_stops_with_vehicle_number.csv")


,stop_date,stop_time,country_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,vehicle_number
0,2020-01-01,0:00:00,Canada,M,59,19,Asian,Drunk Driving,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,UP76DY3473
1,2020-01-01,0:01:00,India,M,35,58,Other,Other,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,RJ83PZ4441
2,2020-01-01,0:02:00,USA,M,26,76,Black,Signal Violation,Speeding,False,Frisk,Ticket,True,16-30 Min,True,RJ32OM7264
3,2020-01-01,0:03:00,Canada,M,26,76,Black,Speeding,DUI,True,Frisk,Warning,False,0-15 Min,True,RJ76TI3807
4,2020-01-01,0:04:00,Canada,M,62,75,Other,Speeding,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,WB63BB8305


In [6]:
import pandas as pd

# Step 1: Load the raw dataset
df = pd.read_csv(r"D:\Guvi-1st Project\traffic_stops - traffic_stops_with_vehicle_number.csv")

# Step 2: Remove columns with all missing values
df.dropna(axis=1, how='all', inplace=True)

# Step 3: Fill missing values
df.fillna({
    'search_type': 'None',
    'is_arrested': False,
    'drugs_related_stop': False,
    'stop_duration': 'Unknown',
    'violation': 'Unknown',
    'stop_outcome': 'Unknown',
    'driver_age': df['driver_age'].median()
}, inplace=True)

# Step 4: Create full 'stopping_date' column from 'stop_date' as datetime (recommended for SQL)
df['stopping_date'] = pd.to_datetime(df['stop_date'], errors='coerce')

# Step 5: Convert 'stop_time' to HH:MM:SS if the column exists
if 'stop_time' in df.columns:
    df['stop_time'] = pd.to_datetime(df['stop_time'], errors='coerce').dt.strftime('%H:%M:%S')
else:
    df['stop_time'] = None

# Step 6: Drop 'driver_age_raw' if it exists
df.drop(columns=['driver_age_raw', 'stop_date'], errors='ignore', inplace=True)

# Step 7: Define expected columns for final export
expected_columns = [
    'stopping_date',
    'stop_time',
    'country_name',
    'driver_gender',
    'driver_age',
    'driver_race',
    'vehicle_number',
    'violation',
    'search_conducted',
    'search_type',
    'stop_outcome',
    'is_arrested',
    'stop_duration',
    'drugs_related_stop'
]

# Step 8: Check for missing columns
missing = [col for col in expected_columns if col not in df.columns]
if missing:
    print("⚠️ Missing columns:", missing)
else:
    df = df[expected_columns]
    print("✅ Data cleaned and formatted for database.")
# Step 8A: Map stop_duration to estimated minutes
duration_map = {
    '0-15 Min': 7.5,
    '16-30 Min': 23,
    '31-45 Min': 38,
    '46-60 Min': 53,
    '60+ Min': 70,
    '90+ Min': 100,
    '120+ Min': 130,
    '>180 Min': 190,
    '1-2 Hours': 90,
    'Unknown': None  # Handle missing/unknowns safely
}

df['stop_duration_minutes'] = df['stop_duration'].map(duration_map)

df.drop(columns=['stop_duration'], inplace=True)

# Step 9: Save cleaned data
df.to_csv("cleaned_traffic_stops.csv", index=False)


C:\Users\mpkol\AppData\Local\Temp\ipykernel_584\3974368847.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\Guvi-1st Project\traffic_stops - traffic_stops_with_vehicle_number.csv")
C:\Users\mpkol\AppData\Local\Temp\ipykernel_584\3974368847.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['stop_time'] = pd.to_datetime(df['stop_time'], errors='coerce').dt.strftime('%H:%M:%S')


✅ Data cleaned and formatted for database.


In [7]:
df.to_csv("cleaned_traffic_stops.csv", index=False)


In [8]:
df.head()

,stopping_date,stop_time,country_name,driver_gender,driver_age,driver_race,vehicle_number,violation,search_conducted,search_type,stop_outcome,is_arrested,drugs_related_stop,stop_duration_minutes
0,2020-01-01,00:00:00,Canada,M,19,Asian,UP76DY3473,Speeding,True,Vehicle Search,Ticket,True,True,23.0
1,2020-01-01,00:01:00,India,M,58,Other,RJ83PZ4441,Other,False,Vehicle Search,Arrest,True,True,23.0
2,2020-01-01,00:02:00,USA,M,76,Black,RJ32OM7264,Speeding,False,Frisk,Ticket,True,True,23.0
3,2020-01-01,00:03:00,Canada,M,76,Black,RJ76TI3807,DUI,True,Frisk,Warning,False,True,7.5
4,2020-01-01,00:04:00,Canada,M,75,Other,WB63BB8305,Other,False,Vehicle Search,Arrest,True,False,23.0


In [5]:
df

,stopping_date,stop_time,country_name,driver_gender,driver_age,driver_race,vehicle_number,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop,stop_duration_minutes
0,2020-01-01,00:00:00,Canada,M,19,Asian,UP76DY3473,Speeding,True,Vehicle Search,Ticket,True,16-30 Min,True,23.0
1,2020-01-01,00:01:00,India,M,58,Other,RJ83PZ4441,Other,False,Vehicle Search,Arrest,True,16-30 Min,True,23.0
2,2020-01-01,00:02:00,USA,M,76,Black,RJ32OM7264,Speeding,False,Frisk,Ticket,True,16-30 Min,True,23.0
3,2020-01-01,00:03:00,Canada,M,76,Black,RJ76TI3807,DUI,True,Frisk,Warning,False,0-15 Min,True,7.5
4,2020-01-01,00:04:00,Canada,M,75,Other,WB63BB8305,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65533,2020-02-15,12:13:00,India,F,48,Black,DL56GW6568,Other,False,Vehicle Search,Arrest,True,16-30 Min,False,23.0
65534,2020-02-15,12:14:00,Canada,F,35,Hispanic,TN73EO7098,Other,True,Vehicle Search,Ticket,False,16-30 Min,True,23.0
65535,2020-02-15,12:15:00,USA,M,41,Asian,GJ33MX8328,DUI,True,Frisk,Ticket,True,30+ Min,True,NaN
65536,2020-02-15,12:16:00,Canada,F,63,Black,KA24UZ8488,Other,False,None,Warning,True,0-15 Min,True,7.5
